In [0]:
# https://drive.google.com/file/d/1vtP3r75yyTQCT-vA92r8k4q8AReBXQLd/view?usp=sharing
# https://drive.google.com/file/d/1LgL0ODKrYmQ5XKdQTTFRFZewtqYHIss-/view?usp=sharing
# https://drive.google.com/file/d/1vLYQZaPFY3ha4iZ5nz2Zth6GABA0aaWJ/view?usp=sharing
# https://drive.google.com/file/d/1VzEdaD-v7st-efUijXLwhc7E3_XjPDCR/view?usp=sharing
# https://drive.google.com/file/d/1Xm2QYEdgDuhJHFScjzap5gKWwGRZGFzC/view?usp=sharing

# https://drive.google.com/file/d/1_ZtXbVbjQqC3Y-O04Jld9O0nXrbuAq01/view?usp=sharing

In [0]:
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
import scipy.io
from sklearn.model_selection import train_test_split
import keras
from keras import optimizers
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Using TensorFlow backend.


In [0]:
def process_data(data):
  x_2d = data['X_2D']
  x_3d = data['X_3D']
  x_3d_transpose = np.transpose(x_3d)
  x_3d = x_3d_transpose 
  t = np.zeros((x_3d.shape[1],x_3d.shape[0],x_3d.shape[2]))
  for i in range(x_3d.shape[0]):
    for j in range(x_3d.shape[1]):
      t[j][i] = x_3d[i][j]
  print(t.shape)
  y = data['categoryLabels']
  y = np.transpose(y)  
  return t,y

In [0]:
ids = ["1vtP3r75yyTQCT-vA92r8k4q8AReBXQLd", "1LgL0ODKrYmQ5XKdQTTFRFZewtqYHIss-", "1vLYQZaPFY3ha4iZ5nz2Zth6GABA0aaWJ", "1VzEdaD-v7st-efUijXLwhc7E3_XjPDCR", "1Xm2QYEdgDuhJHFScjzap5gKWwGRZGFzC"]
k = 0
y = np.array([])
t = np.array([])
for id in ids:
  k+=1
  downloaded = drive.CreateFile({'id':id})
  file_name = 'S' + str(k) + '.mat' 
  downloaded.GetContentFile(file_name)  
  data = scipy.io.loadmat(file_name)
  X,Y = process_data(data)
  if(k==1):
    y = Y
    t = X
  else:
    y = np.concatenate((y,Y),axis=0)
    t = np.concatenate((t,X),axis=1)

(32, 5188, 124)
(32, 5185, 124)
(32, 5186, 124)
(32, 5186, 124)
(32, 5185, 124)


In [0]:
y.shape, t.shape

((25930, 1), (32, 25930, 124))

In [0]:
def perform_PCA(t):
  scaler = StandardScaler()
  h = scaler.fit_transform(t)
  pca = PCA(0.95)
  t_new = pca.fit_transform(h)
  print(t_new.shape)
  return t_new

In [0]:
def show_graph(history):
  plt.plot(history.history['loss'],color='blue',label='Train Loss')
  plt.plot(history.history['val_loss'],color='red',label='Valid Loss')
  plt.title('Loss')
  plt.legend()
  plt.xlabel('No. of epochs')
  plt.ylabel('Loss')
  plt.show()
  plt.plot(history.history['acc'],color='blue',label='Train Accuracy')
  plt.plot(history.history['val_acc'],color='red',label='Valid Accuracy')
  plt.legend()
  plt.title('Accuracy')
  plt.xlabel('No. of epochs')
  plt.ylabel('Accuracy')
  plt.show()

In [0]:
epochs = 50
train_validation_split = 0.1
verbose = 1

In [0]:
y -= 1

In [0]:
seed = 7
np.random.seed(seed)

In [0]:
def run_model(t,y,classes):
  models = []
  histories = []
  df = pd.DataFrame(columns=['Time','Number of features','Train Accuracy','Train Loss','Test Accuracy','Test Loss'])
  for i in range(t.shape[0]):
    print("TIME : " , i)
    x = perform_PCA(t[i])
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    input_dims = x.shape[1]
    train_loss = 0
    train_acc = 0
    test_loss = 0
    test_acc = 0
    k = 0
    for train, test in kfold.split(x, y):
      model = keras.Sequential([])
      if(classes == 6):
        model = keras.Sequential([
          keras.layers.Dense(64,input_dim = input_dims,activation = tf.nn.relu),
          keras.layers.Dense(48,activation = tf.nn.relu),
          keras.layers.Dense(24,activation = tf.nn.relu),
          keras.layers.Dense(16,activation = tf.nn.relu),
          keras.layers.Dense(8, activation = tf.nn.relu),
          keras.layers.Dense(6, activation = 'sigmoid')
        ])
      else:
        model = keras.Sequential([
          keras.layers.Dense(10,input_dim = input_dims,activation = tf.nn.relu),
          # keras.layers.Dense(48,activation = tf.nn.relu),
          # keras.layers.Dense(24,activation = tf.nn.relu),
          # keras.layers.Dense(16,activation = tf.nn.relu),
          # keras.layers.Dense(8, activation = tf.nn.relu),
          keras.layers.Dense(2, activation = 'sigmoid')
        ])
      sgd = optimizers.SGD(lr=1e-3)
      cb = keras.callbacks.EarlyStopping(monitor='val_acc',mode='max',restore_best_weights=True)
      if(classes==6):
        model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
      else:
        model.compile(optimizer='adam',loss='binary_categorical_crossentropy',metrics=['accuracy'])
      history = model.fit(x[train],y[train],validation_split=train_validation_split,epochs=epochs,verbose=verbose,shuffle=True,callbacks=[cb])
      # show_graph(history)
      models.append(model)
      histories.append(history)
      l, a = model.evaluate(x[test],y[test],verbose=verbose)
      test_loss += l
      test_acc += a
      l, a = model.evaluate(x[train],y[train],verbose=verbose)
      train_loss += l
      train_acc += a
    df.loc[i] = [int(i),x.shape[1],train_acc/10.0,train_loss/10.0,test_acc/10.0,test_loss/10.0]
  return df,models,histories

In [0]:
df, models, histories = run_model(t,y,6)
df

TIME :  0
(25930, 73)





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 21000 samples, validate on 2334 samples
Epoch 1/50





21000/21000 [==============================] - 9s 433us/step - loss: 1.7935 - acc: 0.1681 - val_loss: 1.7920 - val_acc: 0.1620
Epoch 2/50
21000/21000 [==============================] - 4s 182us/step - loss: 1.7908 - acc: 0.1758 - val_loss: 1.7914 - val_acc: 0.1714
Epoch 3/50
23334/23334 [==============================] - 1s 49us/step
Train on 21003 samples, validate on 2334 samples
Epoch 1/50
21003/21003 [==============================] - 4s 194us/step - loss: 1.7930 - acc: 0.1641 - val_loss: 1.7917 - val_acc: 0.1667
Epoch 2/50
23337/23337 [==============================] - 1s 51us/step
Train on 21003 samples, validate on 2334 samples
Epoch 1/50
21003/21003 [==============================] - 4s 192us/step - loss: 1.7927 - acc: 0.1619 - val_loss: 1.7919 - val_acc: 0.1680
Epoch 2/50
21003/21003 [======

In [0]:
h1 = np.where(y==[1])
x1 = t[:,h1[0],:]
y1 = np.zeros((h1[0].shape[0]))

h2 = np.where(y==[5])
x2 = t[:,h2[0],:]
y2 = np.ones((h1[0].shape[0]))

print(x1.shape, x2.shape)

t1 = np.concatenate((x1,x2),axis=1)
yhat = np.concatenate((y1,y2),axis=0)
print(t1.shape)
t1 = t1[:,:8640,:]
print(t1.shape)
yhat = yhat[:8640]
print(yhat.shape)

df, models, histories = run_model(t1,yhat,2)
df

In [0]:
h1[0].shape[0]

In [0]:
y[y==1]